In [1]:
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from fairseq.models.roberta import RobertaEncDecModel
import torch
import torch.nn.functional as F

/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-03 04:04:35 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [2]:
path = "weights/nlp"
num_classes = 13
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

phoBERT_cls = RobertaModel.from_pretrained( path + '/PhoBERT_base_fairseq', checkpoint_file='model.pt')

# Load BPE
class BPE():
  bpe_codes = path + '/PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Add header cho classification với số lượng classes = 10
phoBERT_cls.register_classification_head('new_task', num_classes=num_classes)




2022-11-03 04:04:39 | INFO | fairseq.file_utils | loading archive file weights/nlp/PhoBERT_base_fairseq
2022-11-03 04:04:42 | INFO | fairseq.tasks.masked_lm | dictionary: 64000 types
2022-11-03 04:04:45 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 1, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_siz

In [3]:
phoBERT_cls.load_state_dict(torch.load( path + "/phoBert_trained_state_dict/phoBert_state_dict_221101.pth"))
phoBERT_cls.to(device)
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout_module

In [4]:
id2label = ['BHYT', 'address', 'admission_dates', 'age', 'department', 'diagnose',
 'discharge_dates', 'gender', 'hospital_name', 'note', 'patient_name',
 'sign_date', 'treatment']

In [7]:
tokens = 'TTYT Huyện Chư Sê'
token_idxs = phoBERT_cls.encode(tokens)
token_idxs.to(device)
logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
output = F.softmax(logprobs, dim=-1)
cls = output.argmax()
print(output.max())
print(id2label[cls])

tensor(0.7205, device='cuda:0', grad_fn=<MaxBackward1>)
hospital_name


In [8]:
token_idxs

tensor([    0,  4577,  2142, 13906,  2454,     2])